In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scipy.stats as ss

# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [9]:
facebook = pd.read_excel('fb.xlsx')
facebook.set_index('Date')
facebook['OpenTmr'] = facebook['Open '].shift(-1)
facebook['OpenClose']= (facebook['Open ']+ facebook['Close '])/2
facebook['HighLow'] = (facebook['High ']+ facebook['Low '])/2
facebook['OCHL'] = (facebook['Open ']+ facebook['Close ']+facebook['High ']+ facebook['Low '])/4
facebook.head()

Date   Open    High     Low   Close   Volume  OpenTmr  OpenClose  \
0 2016-01-04  104.44  104.44   99.75  102.31  345674   102.33    103.375   
1 2016-01-05  102.33  103.71  101.41  102.80  215313   102.02    102.565   
2 2016-01-06  102.02  103.77  100.75  103.17  234525   102.02    102.595   
3 2016-01-07  102.02  102.02   97.30   97.65  419638    99.00     99.835   
4 2016-01-08   99.00  100.50   97.03   97.18  325973    97.80     98.090   

   HighLow      OCHL  
0  102.095  102.7350  
1  102.560  102.5625  
2  102.260  102.4275  
3   99.660   99.7475  
4   98.765   98.4275

In [10]:
N = facebook.shape[0] # total num days
num_boot = 1# total num bootstrap
T= 250 # start day
window = 200 # training period window
Y =facebook['OpenTmr'][-(N-(T+1)):].values

In [11]:
from sklearn.svm import SVR  


def svm_train(X_train, Y_train, X_pred):  
   
    svr_model = SVR(kernel='rbf', gamma=0.0005)
    result = svr_model.fit(X_train, Y_train)
 
    y_hat = result.predict(X_train)  
    y_pred = result.predict(X_pred)
    rmse_svm=np.sqrt(np.mean((Y_train-y_hat)**2))
    return y_hat, y_pred, rmse_svm

In [12]:
def svm_bstr():
    num_boot = 1
    yhat_svm = np.zeros(N-(T+1))
    for t in range(T+1, N):    
        X_train = facebook[['Close ']][t-window:t-1] 
        X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
        X_pred = facebook[['Close ']][t-1:t]
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
        Y_train = facebook['OpenTmr'][t-window:t-1].values    
        yhat_train = svm_train(X_train, Y_train, X_pred)[0] 
        res_train =  Y_train - yhat_train        
        y_pred_all = np.zeros(num_boot)
        # bootstrap method: switching residuals
        for i in range(0, num_boot):
            err = np.random.choice(res_train, (window-1, ), replace=True)
            y_bstr = yhat_train + err                        
            y_pred_bstr = svm_train(X_train, y_bstr, X_pred)[1]
            y_pred_all[i]=y_pred_bstr
          
        y_pred_svm = y_pred_all.mean() # mean of all bootstrap predictions
        yhat_svm[t-(T+1)]=y_pred_svm # do this for each time step        
    rmse_svm=np.sqrt(np.mean((Y[:-1]-yhat_svm[:-1])**2))
    return yhat_svm, rmse_svm

In [13]:
rmse = svm_bstr()[1]

In [14]:
rmse

5.27296265772958